In [16]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from tf_keras.utils import to_categorical

# Download the full LFW dataset with all people included (no filtering)
lfw_people=fetch_lfw_people(min_faces_per_person=0,resize=0.4,color=False)

# Get the face images and labels
data=lfw_people.images # face images
labels=lfw_people.target # people
target_names= lfw_people.target_names #list of all person names

In [17]:
#Print dataset details
print(f"Total images: {data.shape[0]}")
print(f"Image size : {data.shape[1]}x{data.shape[2]}") # heightxwidth
print(f"Number of classes (people) : {len(target_names)}")

Total images: 13233
Image size : 50x37
Number of classes (people) : 5749


In [18]:
#Reshape the data to include the channel dimension (for grayscale images, channel 1)
data = data.reshape((data.shape[0], data.shape[1],data.shape[2],1))
#split tha data
x_train,x_test, y_train,y_test= train_test_split(data,labels,test_size=0.2,random_state=42)

print(f"Training data shape : {x_train.shape}")
print(f"Testing data shape: {x_test.shape}")

Training data shape : (10586, 50, 37, 1)
Testing data shape: (2647, 50, 37, 1)


In [19]:

import tf_keras as tf
from tf_keras import layers, Model

def mb_conv_block(x,inputs,filters,kernel_size,strides,expand_ratio,se_ratio):
    in_channels=x.shape[-1] # Number of inputs filters
    expanded_filter=in_channels * expand_ratio
    
    #Expension phase
    if expand_ratio != 1:
        x =layers.Conv2D(expanded_filter,kernel_size=1, padding='same',use_bias=False)(x)
        x=layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
    else:
        x=x
        
    #Depthwise convolution
    x=layers.DepthwiseConv2D(kernel_size,strides=strides,padding='same',use_bias=False)(x)
    x=layers.BatchNormalization()(x)
    x=layers.ReLU()(x)
    
    # Squeeze and Excitation block
    # if se_ratio:
    #     se_shape=(1,1,in_channels * expand_ratio)
    #     se=layers.GlobalAveragePooling2D()(x)
    #     se=layers.Reshape(se_shape)(x)
    #     se=layers.Dense(se_shape[-1]//4,activation='relu')(se)
    #     se=layers.Dense(se_shape[-1],activation='sigmoid')(se)
    #     x=layers.multiply([x,se])
    
    
    if se_ratio:
        in_channels = x.shape[-1]
        se_shape = (1, 1, in_channels)
        se = layers.GlobalAveragePooling2D()(x)  # This should output shape (None, in_channels)
        se = layers.Reshape(se_shape)(se)  # Change this line to match the output shape
        se = layers.Dense(in_channels // 4, activation='relu')(se)
        se = layers.Dense(in_channels, activation='sigmoid')(se)
        x = layers.multiply([x, se])  # Apply the squeeze-and-excitation weights

        
    #Output phase
    x=layers.Conv2D(filters,kernel_size=1,padding='same',use_bias=False)(x)
    x= layers.BatchNormalization()(x)
    
    if in_channels ==filters and strides ==1:
        x=layers.add([x,inputs])
        
    return x          
    



In [20]:
def EfficientNetB4(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    
    # Initial Conv layer
    x = layers.Conv2D(48, kernel_size=3, strides=2, padding='same', use_bias=False)(inputs)  # Adjusted filters for B4
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # MBConv Blocks following EfficientNetB4 architecture scaling
    x = mb_conv_block(x, inputs=x, filters=32, kernel_size=3, strides=1, expand_ratio=1, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=32, kernel_size=3, strides=2, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=48, kernel_size=3, strides=2, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=48, kernel_size=3, strides=1, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=80, kernel_size=5, strides=2, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=80, kernel_size=5, strides=1, expand_ratio=6, se_ratio=0.25)
    x = mb_conv_block(x, inputs=x, filters=112, kernel_size=5, strides=1, expand_ratio=6, se_ratio=0.25)  # Adjusted for B4
    x = mb_conv_block(x, inputs=x, filters=112, kernel_size=5, strides=1, expand_ratio=6, se_ratio=0.25)  # Adjusted for B4
    x = mb_conv_block(x, inputs=x, filters=192, kernel_size=3, strides=2, expand_ratio=6, se_ratio=0.25)  # Adjusted for B4
    x = mb_conv_block(x, inputs=x, filters=192, kernel_size=3, strides=1, expand_ratio=6, se_ratio=0.25)  # Adjusted for B4
    x = mb_conv_block(x, inputs=x, filters=320, kernel_size=3, strides=1, expand_ratio=6, se_ratio=0.25)  # Adjusted for B4
    
    # Final layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)  # Adding dropout
    x = layers.Dense(1280, activation='relu')(x)  # Adjusted units for B4
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model_4 = Model(inputs, outputs)
    return model_4

In [21]:
input_shape = (50, 37, 1)  # Set input shape to match the data
model_3 = EfficientNetB4(input_shape=input_shape, num_classes=len(target_names))

In [22]:
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [23]:
# Train the model with the resized data
history_3 = model_3.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/5
331/331 [==============================] - 25s 68ms/step - loss: 8.1304 - accuracy: 0.0402 - val_loss: 9.2284 - val_accuracy: 0.0366
Epoch 2/5
331/331 [==============================] - 22s 65ms/step - loss: 7.6596 - accuracy: 0.0409 - val_loss: 8.7223 - val_accuracy: 0.0366
Epoch 3/5
331/331 [==============================] - 21s 65ms/step - loss: 7.4106 - accuracy: 0.0419 - val_loss: 9.7633 - val_accuracy: 0.0397
Epoch 4/5
331/331 [==============================] - 21s 65ms/step - loss: 7.2649 - accuracy: 0.0439 - val_loss: 9.2522 - val_accuracy: 0.0404
Epoch 5/5
331/331 [==============================] - 22s 65ms/step - loss: 7.1471 - accuracy: 0.0458 - val_loss: 8.7158 - val_accuracy: 0.0400


In [24]:
test_loss, test_acc = model_3.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")

83/83 [==============================] - 1s 15ms/step - loss: 8.7158 - accuracy: 0.0400
Test accuracy: 4.00%
